# Pytorch의 nn.Embedding
- Pytorch의 Embedding Layer는 word2vec과 마찬가지로 word embedding vector를 찾는 **Lookup Table**이다.
    - 단어의 **정수의 고유 index**가 입력으로 들어오면 Embedding Layer의 **그 index의 Vector**를 출력한다.
    - 모델이 학습되는 동안 모델이 풀려는 문제에 맞는 값으로 Embedding Layer의 vector들이 업데이트 된다.
    - Word2Vec의 embedding vector 학습을 nn.Embedding은 자신이 포함된 모델을 학습 하는 과정에서 한다고 생각하면 된다.

In [1]:
import torch
import torch.nn as nn

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


In [2]:
# weight: num_embeddings x embedding_dim
e_layer = nn.Embedding(
    num_embeddings=10,    # embedding할 총 단어수(vocab_size, n_vocab)
    embedding_dim=5,        # embedding vector의 차원수. (한개 단어를 몇개 숫자 vector로 표현)
)

In [5]:
# weight 조회
print(e_layer.weight.shape)
e_layer.weight

torch.Size([10, 5])


Parameter containing:
tensor([[ 1.2872, -0.9666,  1.0572,  1.8553,  1.4034],
        [-0.2536,  0.4192, -2.0518,  0.0535, -1.4707],
        [ 0.3092, -0.5023, -1.0968, -1.5061,  0.2261],
        [-0.7654, -2.6820,  1.2210, -0.9326, -0.6549],
        [ 1.0581,  0.7595,  0.0052,  0.8301, -1.6413],
        [ 0.3635,  0.5969, -0.1236, -1.2737, -0.3104],
        [-1.4983,  1.6032,  0.9305,  1.3301,  0.0593],
        [ 0.0207, -0.5517,  0.8156, -0.9954,  2.0784],
        [ 0.3543,  0.1219, -1.8038,  0.9530,  0.1627],
        [ 0.9938,  1.4051,  0.1349,  0.3557,  1.7975]], requires_grad=True)

In [6]:
# 단어 index 는 정수여야 한다. 
input_data = torch.LongTensor([[3, 6, 1]])    # dtype=int64
result = e_layer(input_data)
result.shape   # [1:문서개수,    3:문서의 토큰수,    5:embedding차원수]

torch.Size([1, 3, 5])

In [8]:
e_layer.weight

Parameter containing:
tensor([[ 1.2872, -0.9666,  1.0572,  1.8553,  1.4034],
        [-0.2536,  0.4192, -2.0518,  0.0535, -1.4707],
        [ 0.3092, -0.5023, -1.0968, -1.5061,  0.2261],
        [-0.7654, -2.6820,  1.2210, -0.9326, -0.6549],
        [ 1.0581,  0.7595,  0.0052,  0.8301, -1.6413],
        [ 0.3635,  0.5969, -0.1236, -1.2737, -0.3104],
        [-1.4983,  1.6032,  0.9305,  1.3301,  0.0593],
        [ 0.0207, -0.5517,  0.8156, -0.9954,  2.0784],
        [ 0.3543,  0.1219, -1.8038,  0.9530,  0.1627],
        [ 0.9938,  1.4051,  0.1349,  0.3557,  1.7975]], requires_grad=True)

In [12]:
result[0, 1]


tensor([-1.4983,  1.6032,  0.9305,  1.3301,  0.0593],
       grad_fn=<SelectBackward0>)

In [ ]:
[-1.4983,  1.6032,  0.9305,  1.3301,  0.0593],

# 네이버 영화 댓글 감성분석(Sentiment Analysis)

## 감성분석(Sentiment Analysis) 이란
입력된 텍스트가 **긍적적인 글**인지 **부정적인**인지 또는 **중립적인** 글인지 분석하는 것을 감성(감정) 분석이라고 한다.   
이를 통해 기업이 고객이 자신들의 기업 또는 제품에 대해 어떤 의견을 가지고 있는지 분석한다.

In [13]:
!pip install korpora

  Using cached Korpora-0.2.0-py3-none-any.whl.metadata (26 kB)
  Using cached dataclasses-0.6-py3-none-any.whl.metadata (3.0 kB)
  Using cached xlrd-2.0.1-py2.py3-none-any.whl.metadata (3.4 kB)
Using cached Korpora-0.2.0-py3-none-any.whl (57 kB)
Using cached dataclasses-0.6-py3-none-any.whl (14 kB)
Using cached xlrd-2.0.1-py2.py3-none-any.whl (96 kB)


## Dataset, DataLoader 생성

### Korpora에서 Naver 영화 댓글 dataset 가져오기
- https://ko-nlp.github.io/Korpora/ko-docs/corpuslist/nsmc.html
- http://github.com/e9t/nsmc/
    - input: 영화댓글
    - output: 0(부정적댓글), 1(긍정적댓글)
#### API
- **corpus 가져오기**
    - `Korpora.load('nsmc')`
- **text/label 조회**
    - `corpus.get_all_texts()` : 전체 corpus의 text들을 tuple로 반환
    - `corpus.get_all_labels()`: 전체 corpus의 label들을 list로 반환
- **train/test set 나눠서 조회**
    - `corpus.train`
    - `corpus.test`
    - `LabeledSentenceKorpusData` 객체에 text와 label들을 담아서 제공.
        - `LabeledSentenceKorpusData.texts`: text들 tuple로 반환.
        - `LabeledSentenceKorpusData.labels`: label들 list로 반환.

In [1]:
from Korpora import Korpora
import pandas as pd

In [3]:
### 데이터 로딩 (다운로딩 + 로딩)
corpus = Korpora.load("nsmc")


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at C:\Users\USER\Korpora\nsmc\ratings_train.txt
[Korpora] Corpus `nsmc` is already installed at C:\Users\USER

In [4]:
type(corpus)

Korpora.korpus_nsmc.NSMCKorpus

In [5]:
# 전체 다 읽어오기
all_text = corpus.get_all_texts()  # (모든 input)
type(all_text), len(all_text)

(tuple, 200000)

In [6]:
print(all_text[0])

아 더빙.. 진짜 짜증나네요 목소리


In [7]:
all_label = corpus.get_all_labels() # list(모든 label)
print(type(all_label), len(all_label))
print(all_label[0])

<class 'list'> 200000
0


In [8]:
## train/test 분리해서 로딩
tr = corpus.train
len(tr.texts), len(tr.labels)

(150000, 150000)

In [9]:
te = corpus.test
# te.texts[0]
len(te.texts), len(te.labels)

(50000, 50000)

# 토큰화 (Tokenization)
1. cleaning (전처리)
2. subword 방식 토큰화

In [10]:
# pip install    jpype1  konlpy

In [26]:
#########################
# cleaning 전처리 - 함수
#########################
from konlpy.tag import Okt
import re
import string

def text_preprocessing(text, tokenizer):
    """
    1. 영문자를 소문자 변경.
    2. 구두점 제거
    3. 토큰화 -> 형태소기반으로 토큰화. 어간을 추출해서 토큰화.  
    # 소문자변환, 어간으로 토큰화 -> 같은 단어는 같은 표현으로 만든다.
    """
    text = text.lower()
    text = re.sub(f"[{string.punctuation}]", ' ', text)
    text = tokenizer.morphs(text, stem=True) # stem=True: 어간 추출
    return " ".join(text)

In [27]:
text = "이 Movie 정말 재미 있어요....,!!!!!!. 꼭보세요.### 저기 있네요."
okt = Okt()
text_preprocessing(text, okt)
# text = text.lower()
# text = re.sub(f"[{string.punctuation}]", ' ', text)
# text = okt.morphs(text, stem=True) # stem=True: 어간 추출
# " ".join(text)  # 리스트의 원소들을 " "을 구분자로 하나의 string으로 합친다.

'이 movie 정말 재미 있다 꼭 보다 저기 있다'

In [32]:
########################
# 전처리
########################
okt = Okt()
all_text = [text_preprocessing(text, okt)  for text in all_text]

In [33]:
all_text[:5]

['아 더빙 진짜 짜증나다 목소리',
 '흠 포스터 보고 초딩 영화 줄 오버 연기 조차 가볍다 않다',
 '너 무재 밓었 다그 래서 보다 추천 한 다',
 '교도소 이야기 구먼 솔직하다 재미 는 없다 평점 조정',
 '사이 몬페 그 의 익살스럽다 연기 가 돋보이다 영화 스파이더맨 에서 늙다 보이다 하다 커스틴 던스트 가 너무나도 이쁘다 보이다']

### 토큰화

In [35]:
# pip install tokenizers

In [36]:
# 전처한 text를 파일로 저장.
corpus_path = "datasets/nsmc_morphs.txt"
with open(corpus_path, 'wt', encoding="utf-8") as fw:
    fw.write('\n'.join(all_text))  
# 리스트안에 모든 원소 문장들을 하나의 string 만들어서 저장.  각 문장 구분자로 enter 추가.

In [37]:
# huggingface tokenizer lib 사용.
## subword 방식 (BPE)
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer

vocab_size = 30000  # 최대 단어수 
min_frequency = 5   # 어휘사전에 포함할 때 최소빈도수 조건.

tk = Tokenizer(
    BPE(unk_token="[UNK]")  # unknown 토큰 - 모르는 토큰을 표시할 토큰.
)
tk.pre_tokenizer = Whitespace()  # 공백을 기준으로 사전 토큰화.
trainer = BpeTrainer(
    vocab_size=vocab_size,
    min_frequency=min_frequency, 
    special_tokens=['[PAD]' , '[UNK]'], # 우리가 등록하는 토큰들. 특정 의미를 가지는 토큰들을 추가
)

In [38]:
# 학습
tk.train([corpus_path], trainer=trainer)

In [40]:
# 토크나이저 저장
tk.save("models/msmc_bpe.json")

# load_tk = Tokenizer.from_file(경로)

In [48]:
##### tokenizer 조회 
# 문자열 token -> 정수 index  (id)   없으면 None
print(tk.token_to_id('김상중'))
# 문자열 token <- 정수 index  (id)
print(tk.id_to_token(16959))
print(tk.id_to_token(30000))  # 없으면 None
print("총 토큰 개수:", tk.get_vocab_size())
# 전체 token 조회
# tk.get_vocab()  # dict-  token : id

None
김상중
None
총 토큰 개수: 22921


In [57]:
### 문서 -> token화 
print(all_text[10])
result = tk.encode(all_text[10])
print(type(result))
print(result)
print(result.tokens)  # 토큰화-token
print(result.ids)       # 토큰화 - id

걍 인피니트 가 짱 이다 진짜 짱 이다 ♥
<class 'tokenizers.Encoding'>
Encoding(num_tokens=9, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
['걍', '인피니트', '가', '짱', '이다', '진짜', '짱', '이다', '♥']
[540, 8863, 506, 2408, 3081, 3096, 2408, 3081, 119]


In [58]:
tk.decode([540, 8863, 506, 2408, 3081, 3096, 2408, 3081, 119])

'걍 인피니트 가 짱 이다 진짜 짱 이다 ♥'

# pytoch Dataset, DataLoader 생성

## Dataset 정의

In [59]:
tk.token_to_id('[PAD]')

0

In [98]:
import torch
from torch.utils.data import Dataset, DataLoader

class NSMCDataset(Dataset):

    def __init__(self, text, label, max_length, tokenizer):
        """
        Parameter
            text: 전체 댓글들 (input)
            label: 전체 정답(긍부정 여부, output)
            max_length: text의 최대 토큰수 (모든 문서의 token수를 max_length에 맞춘다.)
            tokenizer: 토크나이저
        """
        self.text = text
        self.label = label
        self.max_length = max_length
        self.tokenizer = tokenizer

    def __getitem__(self, idx):
        """
        text(댓글) -> 모든 text의 토큰개수를 max_length 에 맞춘다.
        max_length 보다 토큰수가 많은 경우 뒤에를 짤라낸다.
        max_length 보다 적은 경우 모자라는 만큼 뒤에 [PAD] 토큰을 추가.
        """
        
        text = self.text[idx]  # 문장  - "이 영화 재미있네요"
        encode = self.tokenizer.encode(text)  # 문자열 기반으로 토큰화
        text_tokens_id = encode.ids  # [10, 2, 5, 300]
        
        PAD_TOKEN_ID = self.tokenizer.token_to_id('[PAD]')
        seq_length = len(text_tokens_id)  # idx 번째 문서의 토큰 개수
        result = None
        if seq_length >= self.max_length:
            result = text_tokens_id[:self.max_length]
        else: # max_length보다 토큰수가 적은 경우 -> 뒤에 [PAD] 추가.
            result = text_tokens_id + ([PAD_TOKEN_ID] * (self.max_length - seq_length))
       
        label = self.label[idx]
        # return input[idx],  output[idx]  ==> torch.Tensor
        return (torch.tensor(result, dtype=torch.int64), # token id(index) => 정수(int64)
                  torch.tensor([label], dtype=torch.float32))# y: float32
    
    def __len__(self):
        # 전체 데이터 개수를 반환. len(ds객체)
        return len(self.text)

In [74]:
#########################
# Dataset 생성
#########################
okt = Okt()
train_texts = [text_preprocessing(txt, okt) for txt in corpus.train.texts]
test_texts = [text_preprocessing(txt, okt) for txt in corpus.test.texts]

In [99]:
MAX_LENGTH = 40
train_set = NSMCDataset(
    train_texts,            # 댓글들(x)
    corpus.train.labels, # 정답(y)
    max_length = MAX_LENGTH, # 각 문서의 총 토큰개수.
    tokenizer=tk
)

test_set = NSMCDataset(test_texts, corpus.test.labels, 
                                max_length=MAX_LENGTH, tokenizer=tk)

In [100]:
len(train_set), len(test_set)

(150000, 50000)

In [101]:
#### 확인
idx = 10000
x, y = train_set[idx]
print(x.shape, y.shape)
print(x, y)
tk.decode(x.numpy())

torch.Size([40]) torch.Size([1])
tensor([3105, 2203,    4,   16, 3076,  506, 3102,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0]) tensor([1.])


'최고 의 2 d 영화 가 아니다'

In [102]:
################################
# DataLoader
################################
BATCH_SIZE = 64
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE)

In [103]:
# step수
len(train_loader), len(test_loader)

(2343, 782)

In [104]:
x, y = next(iter(train_loader))
x.shape, y.shape
# x shape: [64, 40]  # 64-batch_size(한번 입력되는 문서 개수),  
#                          40: seq_length (한개 문서의 토큰개수)

(torch.Size([64, 40]), torch.Size([64, 1]))

## 모델
- Embedding Layer를 이용해 Word Embedding Vector를 추출한다.
- LSTM을 이용해 Feature 추출
- Linear + Sigmoid로 댓글 긍정일 확률 출력
  
![outline](figures/rnn/RNN_outline.png)

### 모델 정의

In [115]:
import torch
import torch.nn as nn

class NSMCModel(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim, 
                         hidden_size, num_layers, dropout_rate=0):
        """
        Embedding, LSTM, Linear 세개의 Layer를 생성.
        Parameter
            vocab_size: 어휘사전에 등록된 총 단어(토큰) 개수
            embedding_dim: embedding vector의 차원
            hidden_size: LSTM의 hidden state 개수
            num_layers: LSTM의 layer를 몇개 쌓을 것인지. 
            dropout_rate: LSTM의 dropout rate
        """
        super().__init__()
        self.embedding = nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=embedding_dim,
            padding_idx=0  # PAD embedding은 학습하지 않는다.(update안함.)
            # 모자라는 token을 채우는 토큰->padding: [PAD]의 토큰값을 지정
        )
        self.lstm = nn.LSTM(
            input_size=embedding_dim,  # 입력 feature개수 -> nn.Embedding을 통과한 개별 토큰의 원소수
            hidden_size=hidden_size,     # Unit(Node)의 개수.
            num_layers=num_layers, 
            bidirectional=True,             #양방향.(입력된 문서의 내용을 양방향에서 특성을 추출)
            dropout=dropout_rate
        )# batch_first=False  입력/출력 : (seq, batch, features)
        self.dropout = nn.Dropout(dropout_rate) # LSTM->droput->Linear
        
        # classifer(Linear) 입력: many to one: one-문맥정보 (입력 문서의 정보-마지막 timestep 처리결과)
        #                      LSTM (output(o), hidden(x), cellstate(x))
        self.classifier = nn.Linear(
            in_features=hidden_size * 2, # * 2 (양방향)  만약 단방향 * 1
            out_features=1    # 이진분류 => positive일 확률
        )
        self.sigmoid = nn.Sigmoid() # Linear 출력결과(1개) 를 0 ~ 1 확률로 변환.
        
    def forward(self, X):
        # 입력 X shape:   [batch_size,   seq_length:문장토큰수] 
        embeddings = self.embedding(X)
        # embeddings shape : [batch_size, seq_length, embedding_dim]
        # LSTM input shape을 변경: batch_first=False [seq_length, batch_size, embedding_dim]
        embeddings = embeddings.permute(1, 0, 2) # 원소들의 index를 변경.
        output, _ = self.lstm(embeddings) 
        # output shape:   [seq_length:40, batch_size, hidden_size * 2] # 마지막 sequence hidden
        last_output = output[-1, :, :]  # [batch_size, hidden*2]
        last_output = self.dropout(last_output)
        last_output = self.classifier(last_output)
        last_output = self.sigmoid(last_output)
        return last_output
    

In [116]:
tk.get_vocab_size()

22921

In [117]:
# 모델 생성
model = NSMCModel(
    vocab_size=tk.get_vocab_size(), 
    embedding_dim=100,
    hidden_size=32, 
    num_layers=2,
    dropout_rate=0.3
)
print(model)

NSMCModel(
  (embedding): Embedding(22921, 100, padding_idx=0)
  (lstm): LSTM(100, 32, num_layers=2, dropout=0.3, bidirectional=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (classifier): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


In [113]:
x, y = next(iter(train_loader))
x.shape, y.shape

(torch.Size([64, 40]), torch.Size([64, 1]))

In [119]:
yhat = model(x)
yhat.shape

torch.Size([64, 1])

In [120]:
yhat[:5]

tensor([[0.4829],
        [0.4868],
        [0.4831],
        [0.4824],
        [0.4890]], grad_fn=<SliceBackward0>)

## 학습

### Train/Test 함수 정의

In [137]:
import numpy as np
def train_fn(model, dataloader, loss_fn, optimizer, device, interval=100):
    """
    1 epoch 학습 하는 함수.
    interval: 몇 step에 한번씩 로그를 남길지 간격지정.
    """
    # 1. 모델 train 모드로 변환.
    model = model.to(device)
    model.train()
    loss_list = []
    # 2. 학습
    for step, (X, y) in enumerate(dataloader):
        # 1. X, y 를 device 로 이동
        X, y = X.to(device), y.to(device)
        # 2. 추론
        pred = model(X)
        # 3. loss 계산
        loss = loss_fn(pred, y)  # 추론값, 정답
        loss_list.append(loss.item())
        # 3. 학습
        ## grad 계산
        loss.backward()
        ## 파라미터 업데이트
        optimizer.step()
        ## 파라미터 gradient 초기화 
        optimizer.zero_grad()

        if step % interval == 0:
            print(f"\t{step} step Train Loss: {np.mean(loss_list)}") 

In [138]:
### 평가 함수
def test_fn(model, dataloader, loss_fn, device):
    # 1. 모델 mode  변경
    model = model.to(device)
    model.eval()
    loss_list = []
    acc_list = []
    for X, y in dataloader:
        # 2. X, y를 device
        X, y = X.to(device), y.to(device)
        # 추론 - 평가
        with torch.no_grad():
            pred_proba = model(X)  # pos 확률
            pred_label = (pred_proba > 0.5).type(torch.int32)
            loss = loss_fn(pred_proba, y)
            loss_list.append(loss.item())
            # 정확도 
            acc_list.extend(torch.eq(pred_label, y).to("cpu").tolist())     # Tensor -> List
    print(f"\t>>>>>validation loss: {np.mean(loss_list)}, validation accuracy: {np.mean(acc_list)}")      

In [139]:
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [141]:
## 학습
EPOCHS = 3
import time
device = "cuda" if torch.cuda.is_available() else "cpu"
s = time.time()
for epoch in range(EPOCHS):
    print(f"========={epoch}/{EPOCHS}===========")
    train_fn(model, train_loader, loss_fn, optimizer, device, 500)
    test_fn(model, test_loader, loss_fn, device)
e = time.time()
print("학습에 걸린시간: ", (e-s))

=========0/3===========
	0 step Train Loss: 0.4393729567527771
	500 step Train Loss: 0.401001913551085
	1000 step Train Loss: 0.4004125414373396
	1500 step Train Loss: 0.39587123365421284
	2000 step Train Loss: 0.39212915837496654
	>>>>>validation loss: 0.3762972017421442, validation accuracy: 0.83152
=========1/3===========
	0 step Train Loss: 0.36396801471710205
	500 step Train Loss: 0.3424482988026328
	1000 step Train Loss: 0.34713382694449696
	1500 step Train Loss: 0.3502136333674927
	2000 step Train Loss: 0.35284143080120384
	>>>>>validation loss: 0.3755488446377732, validation accuracy: 0.83554
=========2/3===========
	0 step Train Loss: 0.3660000264644623
	500 step Train Loss: 0.3422320996989271
	1000 step Train Loss: 0.33985113539359907
	1500 step Train Loss: 0.34003690503622036
	2000 step Train Loss: 0.3409044269932204
	>>>>>validation loss: 0.38057345765478473, validation accuracy: 0.83644
학습에 걸린시간:  347.86237835884094


### 모델저장

In [144]:
torch.save(model, "models/nsmc_model.pt")

## 추론

In [149]:
#### 전처리
## text cleaning
# 토큰화 + padding 추가.
def pad_tokens(tokens, max_length=40):
    PAD_TOKEN = tk.token_to_id('[PAD]')
    seq_length = len(tokens)
    result = None
    if seq_length >= max_length:
        result = tokens[:max_length]
    else:
        result = tokens + ([PAD_TOKEN] * (max_length - seq_length))
    return result

def text_to_input_tensor(text):
    #1. cleaning
    text = text_preprocessing(text, Okt())
    #2. token화 + padding
    encode = tk.encode(text)
    text_token = pad_tokens(encode.ids)
    return torch.LongTensor(text_token)  # 정수

In [170]:
txt = input("댓글:")
input_data = text_to_input_tensor(txt).unsqueeze(0)  # [40] -> [1, 40]
model = model.to(device)
with torch.no_grad():
    yhat = model(input_data.to(device))

댓글: 돈이 아깝다


In [171]:
yhat.item()

0.012134862132370472

In [172]:
"긍정" if yhat.item() > 0.5 else "부정"

'부정'